# UniLMPrompt

可用的中文预训练参数：[`bert-base`](https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip),[`roberta-wwm-ext-base`](https://drive.google.com/uc?export=download&id=1jMAKIJmPn7kADgD3yQZhpsqM-IRM1qZt),[`roberta-wwm-ext-large`](https://drive.google.com/uc?export=download&id=1dtad0FFzG11CBsawu8hvwwzU2R0FDI94),[`macbert-base`](https://drive.google.com/uc?export=download&id=1aV69OhYzIwj_hn-kO1RiBa-m8QAusQ5b),[`macbert-large`](https://drive.google.com/uc?export=download&id=1lWYxnk1EqTA2Q20_IShxBrCPc5VSDCkT)

In [1]:
import uf

print(uf.__version__)

v2.5.13


# 语言模型

复用 `UniLM` 的 MASK 采样能力

In [2]:
model = uf.UniLMPrompt("../../ref/bert_config.json", "../../ref/vocab.txt")
print(model)

INFO:tensorflow:LM Mode: `bi`. Use method `.to_mode()` to convert it into `bi`, `l2r`, `r2l` or `s2s`.
uf.UniLMPrompt(
    config_file="../../ref/bert_config.json",
    vocab_file="../../ref/vocab.txt",
    max_seq_length=128,
    init_checkpoint=None,
    output_dir=None,
    gpu_ids=None,
    drop_pooler=False,
    max_predictions_per_seq=20,
    masked_lm_prob=0.15,
    short_seq_prob=0.1,
    do_whole_word_mask=False,
    mode="bi",
    do_lower_case=True,
    truncate_method="LIFO",
)


In [3]:
# bi/l2r/r2l模式下
model.to_mode("bi")
X_tokenized = [
     [
          ["[Soft Prompt]"],                                            # soft prompt
          model.tokenizer.tokenize("被掩码的文字是？"),                    # task description
          model.tokenizer.tokenize("天亮以前说再见, 笑着泪流满面"),         # 文本
     ],
     [
          ["[Soft Prompt]"],                        
          model.tokenizer.tokenize("被掩码的文字是？"),                    
          model.tokenizer.tokenize("他想知道那是谁, 为何总沉默寡言"),
     ],
]

# s2s模式下
# model.to_mode("s2s")
# X_tokenized = [
#      [
#           ["[Soft Prompt]"],                                            # soft prompt
#           model.tokenizer.tokenize("生成下一句歌词"),                     # task description
#           model.tokenizer.tokenize("天亮以前说再见"),                     # 原始文本
#           model.tokenizer.tokenize("笑着泪流满面"),                       # 生成文本
#      ],
#      [
#           ["[Soft Prompt]"],                        
#           model.tokenizer.tokenize("生成下一句歌词"),                     # task description
#           model.tokenizer.tokenize("他想知道那是谁"),                     # 原始文本
#           model.tokenizer.tokenize("为何总沉默寡言"),                     # 生成文本
#      ],
# ]

y = None

In [4]:
model.fit(X_tokenized=X_tokenized, total_steps=20)

/Users/geyingli/Library/Python/3.8/lib/python/site-packages/uf-2.5.13-py3.8.egg/uf/apps/bert/bert.py:235: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  query_layer = tf.layers.dense(
/Users/geyingli/Library/Python/3.8/lib/python/site-packages/uf-2.5.13-py3.8.egg/uf/apps/bert/bert.py:245: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  key_layer = tf.layers.dense(
/Users/geyingli/Library/Python/3.8/lib/python/site-packages/uf-2.5.13-py3.8.egg/uf/apps/bert/bert.py:255: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  value_layer = tf.layers.dense(
/Users/geyingli/Library/Python/3.8/lib/python/site-packages/uf-2.5.13-py3.8.egg/uf/apps/bert/bert.py:379: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Ple

INFO:tensorflow:Build graph with 307,461,528 parameters (among which 102,880,904 are trainable)
INFO:tensorflow:Running local_init_op
INFO:tensorflow:Done running local_init_op
INFO:tensorflow:Running training on 2 samples (step 0 -> 20)
INFO:tensorflow:step 1, MLM accuracy 0.0000, MLM loss 1.018881, 0.14 steps/sec, 0.27 examples/sec
INFO:tensorflow:step 2, MLM accuracy 0.7500, MLM loss 0.767338, 0.51 steps/sec, 1.02 examples/sec
INFO:tensorflow:step 3, MLM accuracy 1.0000, MLM loss 0.587027, 0.49 steps/sec, 0.99 examples/sec
INFO:tensorflow:step 4, MLM accuracy 1.0000, MLM loss 0.479584, 0.52 steps/sec, 1.04 examples/sec
INFO:tensorflow:step 5, MLM accuracy 1.0000, MLM loss 0.403024, 0.52 steps/sec, 1.04 examples/sec
INFO:tensorflow:step 6, MLM accuracy 1.0000, MLM loss 0.331306, 0.49 steps/sec, 0.98 examples/sec
INFO:tensorflow:step 7, MLM accuracy 1.0000, MLM loss 0.270946, 0.52 steps/sec, 1.05 examples/sec
INFO:tensorflow:step 8, MLM accuracy 1.0000, MLM loss 0.218453, 0.53 steps/s

In [5]:
X_tokenized = [
     [
          ["[Soft Prompt]"],                           
          model.tokenizer.tokenize("被掩码的文字是？"),
          ["他", "想", "[MASK]", "道", "那", "是", "谁"],        # 手动赋予Mask (与训练阶段采样不同，因此此case预测错误正常)
     ],
]
model.predict(X_tokenized=X_tokenized)

INFO:tensorflow:Running inference on 1 samples
INFO:tensorflow:process 100.0%, 0.88 examples/sec


{'mlm_preds': [['前']]}

# 下游任务

大部分 NLP 任务都能够通过某种形式，转换为语言模型任务

In [6]:
model = uf.UniLMPrompt("../../ref/bert_config.json", "../../ref/vocab.txt", masked_lm_prob=0)
print(model)

INFO:tensorflow:LM Mode: `bi`. Use method `.to_mode()` to convert it into `bi`, `l2r`, `r2l` or `s2s`.
uf.UniLMPrompt(
    config_file="../../ref/bert_config.json",
    vocab_file="../../ref/vocab.txt",
    max_seq_length=128,
    init_checkpoint=None,
    output_dir=None,
    gpu_ids=None,
    drop_pooler=False,
    max_predictions_per_seq=20,
    masked_lm_prob=0,
    short_seq_prob=0.1,
    do_whole_word_mask=False,
    mode="bi",
    do_lower_case=True,
    truncate_method="LIFO",
)


In [7]:
X_tokenized = [
     [
          ["[Soft Prompt]"],                                                 # soft prompt
          ["情", "民"],                                                       # verbalizer prompt
          model.tokenizer.tokenize("下面这句歌词来自于") + ["[MASK]", "歌"],    # task description
          model.tokenizer.tokenize("天亮以前说再见, 笑着泪流满面"),              # 文本
     ],
     [
          ["[Soft Prompt]"],                        
          ["情", "民"],                                                     
          model.tokenizer.tokenize("下面这句歌词来自于") + ["[MASK]", "歌"],          
          model.tokenizer.tokenize("在天的尽头, 与月亮把盏"),
     ],
]
y = [["情"], ["民"]]

In [8]:
model.fit(X_tokenized=X_tokenized, y=y, total_steps=20)

INFO:tensorflow:Build graph with 307,461,528 parameters (among which 102,880,904 are trainable)
INFO:tensorflow:Running local_init_op
INFO:tensorflow:Done running local_init_op
INFO:tensorflow:Running training on 2 samples (step 0 -> 20)
INFO:tensorflow:step 1, MLM accuracy 0.0000, MLM loss 0.531528, 0.13 steps/sec, 0.27 examples/sec
INFO:tensorflow:step 2, MLM accuracy 1.0000, MLM loss 0.357832, 0.56 steps/sec, 1.11 examples/sec
INFO:tensorflow:step 3, MLM accuracy 0.5000, MLM loss 0.275157, 0.56 steps/sec, 1.11 examples/sec
INFO:tensorflow:step 4, MLM accuracy 0.5000, MLM loss 0.221405, 0.55 steps/sec, 1.09 examples/sec
INFO:tensorflow:step 5, MLM accuracy 1.0000, MLM loss 0.189745, 0.55 steps/sec, 1.10 examples/sec
INFO:tensorflow:step 6, MLM accuracy 0.5000, MLM loss 0.179693, 0.54 steps/sec, 1.09 examples/sec
INFO:tensorflow:step 7, MLM accuracy 0.5000, MLM loss 0.150096, 0.55 steps/sec, 1.10 examples/sec
INFO:tensorflow:step 8, MLM accuracy 0.5000, MLM loss 0.143473, 0.56 steps/s

In [9]:
model.predict(X_tokenized=X_tokenized)

INFO:tensorflow:Running inference on 2 samples
INFO:tensorflow:process 100.0%, 1.65 examples/sec


{'mlm_preds': [['情'], ['民']]}